> INVESTING.COM 에서  NASDAQ, S&P, CBOE, 환율, 2년 만기 미국채 선물, 10년 만기 미국채 선물 가져오기

# API

In [7]:
import FinanceDataReader as fdr
import pandas as pd
from datetime import date, timedelta

In [11]:
toay = date.today().strftime('%Y%m%d')
yday = (date.today()-timedelta(1)).strftime('%Y%m%d')
bfyday = (date.today() - timedelta(2)).strftime('%Y%m%d')

In [3]:
nasdaq = fdr.DataReader('NASDAQCOM', start='2021-09-28', data_source='fred')
nasdaq

,NASDAQCOM
DATE,
2021-09-28,14546.68
2021-09-29,14512.44
2021-09-30,14448.58
2021-10-01,14566.70
2021-10-04,14255.48
2021-10-05,14433.83


In [4]:
sp500 = fdr.DataReader('US500', "20210927", "20210928")
sp500

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2021-09-27,4443.11,4442.12,4457.30,4436.19,0.0,-0.0028
2021-09-28,4352.63,4419.54,4419.54,4346.33,0.0,-0.0204


In [12]:
try: 
    cboe = fdr.DataReader('VIX', yday, tday)[['Close']] # S&P 500 VIX
except: 
    print('저녁에 업데이트 됩니다.')
    cboe = pd.DataFrame(columns=['close'])
cboe

저녁에 업데이트 됩니다.


,close


In [13]:
rate = fdr.DataReader('USD/KRW',  yday, yday)
rate

,Close,Open,High,Low,Change
Date,,,,,
2021-10-06,1190.21,1186.69,1197.94,1185.9,0.0031


In [14]:
tmp = rate.reset_index()
tmp

,Date,Close,Open,High,Low,Change
0,2021-10-06,1190.21,1186.69,1197.94,1185.9,0.0031


## 함수화

In [16]:
import pandas as pd
import sqlite3
import FinanceDataReader as fdr
from datetime import date, timedelta

class INVESTING:
    def getDate(self):
        self.yday = (date.today() - timedelta(1)).strftime('%Y%m%d')  # 1일 전
        self.bfyday = (date.today() - timedelta(2)).strftime('%Y%m%d')  # 2일 전
        
    def getStock(self):
        self.sp500 = fdr.DataReader('US500',self.yday, self.yday)
        self.sp500 = self.sp500[['Close']]

        self.cboe = fdr.DataReader('VIX', self.yday, self.yday)
        self.cboe = self.cboe[['Close']]

        self.rate = fdr.DataReader('USD/KRW', self.yday, self.yday)
        self.rate  = self.rate[['Close']]

    def saving(self):
        self.df = pd.concat([self.sp500, self.cboe, self.rate], axis=1)
        self.df_invest = self.df.reset_index()
        self.df_invest.columns=['date', 'sp', 'cboe', 'exchangerate']
        self.df_invest = self.df_invest.set_index('date')
        print(self.df_invest)
#         self.df.to_sql('samsung', conn, if_exists='append')
#         conn.commit()  # db에 저장

if __name__ == "__main__":
    
    invest = INVESTING()
    invest.getDate()
    invest.getStock()
    invest.saving()

                 sp  cboe  exchangerate
date                                   
2021-10-06  4363.55  21.0       1190.21


# Scraping

In [33]:
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from pykrx import stock
import FinanceDataReader as fdr
from datetime import date, datetime, timedelta
import schedule
import time


class UpdateDB:
    def __init__(self):
        self.yday = (date.today() - timedelta(1)).strftime('%Y%m%d')  # 1일 전
        self.bfyday = (date.today() - timedelta(2)).strftime('%Y%m%d')  # 2일 전
        self.options = Options()
        self.options.add_argument('headless')
        self.contents = []

    def getINVEST(self):  # Investing.com 데이터
        self.sp500 = fdr.DataReader('US500', self.yday, self.yday)[['Close']]
        self.cboe = fdr.DataReader('VIX', self.yday, self.yday)[['Close']]
        self.rate = fdr.DataReader('USD/KRW', self.yday, self.yday)[['Close']]
        # 거시경제지표 중 S&P500은 데이터 업데이트가 늦음
        df_index = pd.concat([self.sp500, self.cboe, self.rate], axis=1).reset_index()
        df_index.columns = ['date', 'sp', 'cboe', 'exchangerate']
        self.df_index = df_index.set_index('date')
        print('getINVEST complete.')
        print(self.df_index)
        return self.df_index

    def crawlINVEST(self):  # Investing.com 크롤링
        self.getINVEST()
        self.urls = ['https://kr.investing.com/indices/nasdaq-composite-historical-data','https://kr.investing.com/rates-bonds/us-2-yr-t-note-historical-data', 'https://kr.investing.com/rates-bonds/us-10-yr-t-note-historical-data']
        for url in self.urls:
            self.driver = webdriver.Chrome('C:\chromedriver.exe', options=self.options)
            self.driver.get(url)
            self.soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            try:
                self.index = self.soup.select('#last_last')
            except:
                self.driver.execute_script("window.scrollTo(0, 700)")
                self.index = self.soup.select('#curr_table > tbody > tr:nth-child(1) > td:nth-child(2)')
            self.contents.append(self.index[0].text.strip())
            del self.soup
            self.driver.quit()
        self.dateindex = pd.DatetimeIndex([(date.today() - timedelta(1))])  #datetime.today().date()
        df_crawl = pd.DataFrame([self.contents], index= self.dateindex, columns=['nasdaq','futures2y', 'futures10y']).reset_index().rename(columns={'index': 'date'})
        self.df_crawl = df_crawl.set_index('date')
        print('crawlINVEST complete.')
        print(self.df_crawl)
        return self.df_crawl

    def mergeINVEST(self):
        self.crawlINVEST()
        self.df_invest = pd.merge(self.df_index, self.df_crawl, on='date')
        print(self.df_invest)
        return self.df_invest
    
    
if __name__ == "__main__":
    updatedb = UpdateDB()  # 클래스 선언
    updatedb.mergeINVEST()   # sp, cboe, exchangerate, nasdaq, futures2y, futures10y


getINVEST complete.
                 sp  cboe  exchangerate
date                                   
2021-10-06  4363.55  21.0       1190.21
crawlINVEST complete.
              nasdaq futures2y futures10y
date                                     
2021-10-06  14,501.9    110.01     131.73
                 sp  cboe  exchangerate    nasdaq futures2y futures10y
date                                                                  
2021-10-06  4363.55  21.0       1190.21  14,501.9    110.01     131.73


### 2년 만기 미국채 선물

In [ ]:
content = []

In [ ]:
# webdirver 설정

driver = webdriver.Chrome('C:\chromedriver.exe')
driver.get('https://kr.investing.com/rates-bonds/us-2-yr-t-note-historical-data')
driver.set_window_size(1080,1080)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

us2yr = soup.select('#curr_table > tbody > tr:nth-child(1) > td:nth-child(2)')
us2yr[0].text.strip()
content.append(us2yr[0].text.strip())

del soup
driver.close()

### 10년 만기 미국채 선물

In [ ]:
driver = webdriver.Chrome('C:\chromedriver.exe')
driver.get('https://kr.investing.com/rates-bonds/us-10-yr-t-note-historical-data')
driver.set_window_size(1080,1080)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

us10yr = soup.select('#curr_table > tbody > tr:nth-child(1) > td:nth-child(2)')
us10yr[0].text.strip()
content.append(us10yr[0].text.strip())

del soup
driver.close()

In [ ]:
content

### 날짜 인덱스 부여

In [ ]:
from datetime import date, datetime, timedelta

#date = datetime.today().replace(hour=0, minute=0, second=0)
#date = pd.DatetimeIndex([date.strftime('%Y-%m-%d %H:%M:%S')])
date = pd.DatetimeIndex([datetime.today().date()])
date

In [ ]:
df = pd.DataFrame([content], date, ['futures2y','futures10y'])
df = df.reset_index().rename(columns={'index': 'date'})
df = df.set_index('date')
df

## 함수화

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date, datetime, timedelta


class INDEXCRAWL:
    def __init__(self):
        self.contents = []
    
    def crawling(self,url):
        self.driver = webdriver.Chrome('C:\chromedriver.exe')
        self.driver.get(url)
        self.driver.set_window_size(720,720) 
        #self.driver.execute_script("window.scrollTo(0, 700)")
        
        self.soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        self.index = self.soup.select('#last_last')
        #self.index = self.soup.select('#curr_table > tbody > tr:nth-child(1) > td:nth-child(2)')
        self.contents.append(self.index[0].text.strip())
        
        del self.soup
        self.driver.quit()
        
        
    def saving(self):
        self.date = pd.DatetimeIndex([datetime.today().date()])
        self.df = pd.DataFrame([self.contents], self.date, ['futures2y','futures10y'])
        print(self.df)
        
    

if __name__ == "__main__":
    crawl = INDEXCRAWL()
    us2yr ='https://kr.investing.com/rates-bonds/us-2-yr-t-note-historical-data'
    crawl.crawling(us2yr)
    us10yr = 'https://kr.investing.com/rates-bonds/us-10-yr-t-note-historical-data'
    crawl.crawling(us10yr)
    crawl.saving()